# 🏦 BVMT Stock Price Prediction with LSTM
## IHEC-CODELAB 2.0 - Deep Learning Pipeline

**Objective:** Predict the next 5 days closing prices for BVMT stocks using LSTM

**Model:** LSTM (Long Short-Term Memory) - captures temporal dependencies

**Metrics:** RMSE, MAE, Directional Accuracy

---

### 📋 Table of Contents
1. Setup & Data Loading
2. Data Exploration & Cleaning
3. Feature Engineering
4. Sequence Preparation for LSTM
5. Model Architecture & Training
6. Evaluation & Visualization
7. Export for Production

## 1. 📦 Setup & Data Loading

In [ ]:
# Install required packages (run in Colab)
!pip install -q tensorflow pandas numpy scikit-learn matplotlib seaborn ta

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
import gc
warnings.filterwarnings('ignore')

# TensorFlow/Keras imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, Input, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import mixed_precision

# === COLAB GPU OPTIMIZATIONS ===
# Enable mixed precision for faster training & lower memory
mixed_precision.set_global_policy('mixed_float16')

# Enable memory growth to prevent OOM
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    print(f"✅ GPU memory growth enabled for {len(gpus)} GPU(s)")

# Set seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Set plot style
plt.style.use('dark_background')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 12

print(f"✅ TensorFlow version: {tf.__version__}")
print(f"✅ GPU available: {tf.config.list_physical_devices('GPU')}")
print(f"✅ Mixed precision: {mixed_precision.global_policy().name}")
print("✅ Libraries imported successfully!")

In [ ]:
# 📁 Upload your data files to Colab
from google.colab import files
uploaded = files.upload()
print(f"✅ Uploaded {len(uploaded)} files")

In [ ]:
def load_txt_file(filepath):
    """Load 2016-2021 fixed-width text files."""
    df = pd.read_fwf(
        filepath,
        skiprows=2,
        header=None,
        names=['SEANCE', 'GROUPE', 'CODE', 'VALEUR', 'OUVERTURE', 'CLOTURE', 
               'PLUS_BAS', 'PLUS_HAUT', 'QUANTITE_NEGOCIEE', 'NB_TRANSACTION', 'CAPITAUX', 'IND_RES'],
        encoding='latin-1'
    )
    if 'IND_RES' in df.columns:
        df = df.drop('IND_RES', axis=1)
    return df

def load_csv_file(filepath):
    """Load 2022-2025 CSV files."""
    df = pd.read_csv(filepath, sep=';', encoding='utf-8')
    df.columns = df.columns.str.strip()
    return df

def load_all_data():
    """Load and combine all data files."""
    all_dfs = []
    
    for year in range(2016, 2022):
        try:
            df = load_txt_file(f'histo_cotation_{year}.txt')
            df['source_year'] = year
            all_dfs.append(df)
            print(f"✅ Loaded {year}: {len(df)} rows")
        except Exception as e:
            print(f"⚠️ Could not load {year}: {e}")
    
    for year in range(2022, 2026):
        try:
            df = load_csv_file(f'histo_cotation_{year}.csv')
            df['source_year'] = year
            all_dfs.append(df)
            print(f"✅ Loaded {year}: {len(df)} rows")
        except Exception as e:
            print(f"⚠️ Could not load {year}: {e}")
    
    combined = pd.concat(all_dfs, ignore_index=True)
    print(f"\n📊 Total combined: {len(combined)} rows")
    return combined

raw_data = load_all_data()

## 2. 🔍 Data Exploration & Cleaning

In [ ]:
print("📋 Dataset Info:")
print(f"Shape: {raw_data.shape}")
print(f"\nColumns: {raw_data.columns.tolist()}")
raw_data.head(10)

In [ ]:
def clean_data(df):
    """Clean and standardize the BVMT dataset."""
    df = df.copy()
    
    def parse_date(x):
        if pd.isna(x):
            return pd.NaT
        x = str(x).strip()
        for fmt in ['%d/%m/%Y', '%Y-%m-%d', '%d-%m-%Y']:
            try:
                return pd.to_datetime(x, format=fmt)
            except:
                continue
        return pd.NaT
    
    df['date'] = df['SEANCE'].apply(parse_date)
    
    df['GROUPE'] = pd.to_numeric(df['GROUPE'], errors='coerce')
    df = df[df['GROUPE'] == 11].copy()
    print(f"✅ Filtered to main market: {len(df)} rows")
    
    numeric_cols = ['OUVERTURE', 'CLOTURE', 'PLUS_BAS', 'PLUS_HAUT', 
                    'QUANTITE_NEGOCIEE', 'NB_TRANSACTION', 'CAPITAUX']
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    df = df.rename(columns={
        'VALEUR': 'stock_name',
        'CODE': 'stock_code',
        'OUVERTURE': 'open',
        'CLOTURE': 'close',
        'PLUS_BAS': 'low',
        'PLUS_HAUT': 'high',
        'QUANTITE_NEGOCIEE': 'volume',
        'NB_TRANSACTION': 'transactions',
        'CAPITAUX': 'capital'
    })
    
    df = df.dropna(subset=['date'])
    df = df[(df['close'] > 0) & df['close'].notna()]
    df = df[df['volume'] > 0].copy()
    print(f"✅ After removing invalid data: {len(df)} rows")
    
    df = df.sort_values(['stock_name', 'date']).reset_index(drop=True)
    return df

data = clean_data(raw_data)
print(f"\n📊 Cleaned dataset shape: {data.shape}")

In [ ]:
# Select top liquid stocks
stock_stats = data.groupby('stock_name').agg({
    'volume': ['count', 'sum'],
    'close': ['mean', 'std'],
    'date': ['min', 'max']
}).round(2)

stock_stats.columns = ['trading_days', 'total_volume', 'avg_price', 'price_std', 'first_date', 'last_date']
stock_stats = stock_stats.sort_values('total_volume', ascending=False)

TOP_N_STOCKS = 15
selected_stocks = stock_stats.head(TOP_N_STOCKS).index.tolist()
data = data[data['stock_name'].isin(selected_stocks)].copy()

print(f"📊 Selected {TOP_N_STOCKS} stocks with {len(data)} total records")
print(f"\nStocks: {selected_stocks}")

## 3. 🛠️ Feature Engineering

Same features as XGBoost for fair comparison.

In [ ]:
def create_features(df):
    """Create technical indicators and features for LSTM."""
    df = df.copy().sort_values('date').reset_index(drop=True)
    
    # Returns
    df['return_1d'] = np.log(df['close'] / df['close'].shift(1))
    df['return_5d'] = np.log(df['close'] / df['close'].shift(5))
    df['return_10d'] = np.log(df['close'] / df['close'].shift(10))
    df['return_20d'] = np.log(df['close'] / df['close'].shift(20))
    
    # Volatility
    df['volatility_5d'] = df['return_1d'].rolling(5).std()
    df['volatility_10d'] = df['return_1d'].rolling(10).std()
    df['volatility_20d'] = df['return_1d'].rolling(20).std()
    
    # Moving Averages
    for window in [5, 10, 20, 50]:
        df[f'sma_{window}'] = df['close'].rolling(window).mean()
        df[f'price_to_sma_{window}'] = df['close'] / df[f'sma_{window}']
    
    # EMA
    df['ema_12'] = df['close'].ewm(span=12).mean()
    df['ema_26'] = df['close'].ewm(span=26).mean()
    df['price_to_ema_12'] = df['close'] / df['ema_12']
    
    # Volume features
    df['volume_sma_5'] = df['volume'].rolling(5).mean()
    df['volume_sma_20'] = df['volume'].rolling(20).mean()
    df['volume_ratio'] = df['volume'] / df['volume_sma_20']
    df['volume_change'] = df['volume'].pct_change()
    
    # RSI
    delta = df['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['rsi_14'] = 100 - (100 / (1 + rs))
    
    # MACD
    df['macd'] = df['ema_12'] - df['ema_26']
    df['macd_signal'] = df['macd'].ewm(span=9).mean()
    df['macd_hist'] = df['macd'] - df['macd_signal']
    
    # Bollinger Bands
    df['bb_middle'] = df['close'].rolling(20).mean()
    df['bb_std'] = df['close'].rolling(20).std()
    df['bb_upper'] = df['bb_middle'] + 2 * df['bb_std']
    df['bb_lower'] = df['bb_middle'] - 2 * df['bb_std']
    df['bb_width'] = (df['bb_upper'] - df['bb_lower']) / df['bb_middle']
    df['bb_position'] = (df['close'] - df['bb_lower']) / (df['bb_upper'] - df['bb_lower'])
    
    # Price range
    df['intraday_range'] = (df['high'] - df['low']) / df['close']
    df['gap_open'] = (df['open'] - df['close'].shift(1)) / df['close'].shift(1)
    
    # Time features
    df['day_of_week'] = df['date'].dt.dayofweek
    df['month'] = df['date'].dt.month
    df['quarter'] = df['date'].dt.quarter
    
    return df

# Apply to each stock
print("🛠️ Creating features...")
featured_data = []
for stock in selected_stocks:
    stock_df = data[data['stock_name'] == stock].copy()
    stock_df = create_features(stock_df)
    featured_data.append(stock_df)
    print(f"  ✅ {stock}: {len(stock_df)} rows")

data_with_features = pd.concat(featured_data, ignore_index=True)
print(f"\n📊 Total: {data_with_features.shape}")

In [ ]:
# Define feature columns (same as XGBoost)
FEATURE_COLS = [
    'return_1d', 'return_5d', 'return_10d', 'return_20d',
    'volatility_5d', 'volatility_10d', 'volatility_20d',
    'price_to_sma_5', 'price_to_sma_10', 'price_to_sma_20', 'price_to_sma_50',
    'price_to_ema_12',
    'volume_ratio', 'volume_change',
    'rsi_14', 'macd', 'macd_signal', 'macd_hist',
    'bb_width', 'bb_position',
    'intraday_range', 'gap_open',
    'day_of_week', 'month', 'quarter'
]

print(f"📋 Using {len(FEATURE_COLS)} features")

## 4. 🔄 Sequence Preparation for LSTM

LSTM requires sequences of data. We'll use a sliding window approach:
- **Input**: Past N days of features
- **Output**: Next 5 days returns (Day 1, Day 2, Day 3, Day 4, Day 5)

In [ ]:
# LSTM Hyperparameters
SEQUENCE_LENGTH = 60  # Look back 60 trading days (~3 months)
PREDICTION_HORIZONS = [1, 2, 3, 4, 5]  # Predict 5 days ahead

print(f"⚙️ LSTM Configuration:")
print(f"  Sequence length: {SEQUENCE_LENGTH} days")
print(f"  Prediction horizons: {PREDICTION_HORIZONS}")
print(f"  Features per timestep: {len(FEATURE_COLS)}")

In [ ]:
def create_targets(df, horizons=[1, 2, 3, 4, 5]):
    """Create target variables for multi-day prediction."""
    df = df.copy()
    for h in horizons:
        df[f'target_return_{h}d'] = (df['close'].shift(-h) - df['close']) / df['close']
        df[f'target_price_{h}d'] = df['close'].shift(-h)
    return df

# Apply targets per stock
print("🎯 Creating targets...")
final_data = []
for stock in selected_stocks:
    stock_df = data_with_features[data_with_features['stock_name'] == stock].copy()
    stock_df = create_targets(stock_df)
    final_data.append(stock_df)

df_final = pd.concat(final_data, ignore_index=True)
df_final = df_final.dropna()
print(f"📊 Final dataset: {df_final.shape}")

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

def prepare_sequences_per_stock(df, feature_cols, sequence_length, horizons):
    """
    Prepare sequences for LSTM training per stock.
    
    Returns:
        X: (samples, sequence_length, features)
        y: (samples, num_horizons) - returns for day 1-5
        close_prices: (samples,) - current close price for each sample
        dates: (samples,) - dates for each sample
    """
    X_all, y_all = [], []
    close_prices_all, dates_all = [], []
    
    for stock in df['stock_name'].unique():
        stock_df = df[df['stock_name'] == stock].sort_values('date').reset_index(drop=True)
        
        if len(stock_df) < sequence_length + max(horizons):
            continue
        
        # Get features and targets
        features = stock_df[feature_cols].values
        target_cols = [f'target_return_{h}d' for h in horizons]
        targets = stock_df[target_cols].values
        close = stock_df['close'].values
        dates = stock_df['date'].values
        
        # Create sequences
        for i in range(sequence_length, len(stock_df) - max(horizons)):
            X_all.append(features[i-sequence_length:i])
            y_all.append(targets[i])
            close_prices_all.append(close[i])
            dates_all.append(dates[i])
    
    return np.array(X_all), np.array(y_all), np.array(close_prices_all), np.array(dates_all)

# Prepare sequences
print("🔄 Preparing sequences...")
X, y, close_prices, dates = prepare_sequences_per_stock(
    df_final, FEATURE_COLS, SEQUENCE_LENGTH, PREDICTION_HORIZONS
)

print(f"\n📊 Sequence shapes:")
print(f"  X: {X.shape} (samples, timesteps, features)")
print(f"  y: {y.shape} (samples, horizons)")

In [ ]:
# Scale features
# For LSTM, we scale each feature across the entire dataset

# Reshape for scaling: (samples * timesteps, features)
n_samples, n_timesteps, n_features = X.shape
X_flat = X.reshape(-1, n_features)

# Fit scaler
feature_scaler = StandardScaler()
X_flat_scaled = feature_scaler.fit_transform(X_flat)

# Reshape back to sequences
X_scaled = X_flat_scaled.reshape(n_samples, n_timesteps, n_features)

print(f"✅ Features scaled: {X_scaled.shape}")

# Scale targets (returns) - optional but can help
target_scaler = StandardScaler()
y_scaled = target_scaler.fit_transform(y)

print(f"✅ Targets scaled: {y_scaled.shape}")

In [ ]:
# Temporal train/val/test split
# Sort by date and split
sort_idx = np.argsort(dates)
X_scaled = X_scaled[sort_idx]
y_scaled = y_scaled[sort_idx]
y = y[sort_idx]
close_prices = close_prices[sort_idx]
dates = dates[sort_idx]

n = len(X_scaled)
train_end = int(n * 0.7)
val_end = int(n * 0.85)

X_train, X_val, X_test = X_scaled[:train_end], X_scaled[train_end:val_end], X_scaled[val_end:]
y_train, y_val, y_test = y_scaled[:train_end], y_scaled[train_end:val_end], y_scaled[val_end:]
y_train_raw, y_val_raw, y_test_raw = y[:train_end], y[train_end:val_end], y[val_end:]
close_train, close_val, close_test = close_prices[:train_end], close_prices[train_end:val_end], close_prices[val_end:]
dates_train, dates_val, dates_test = dates[:train_end], dates[train_end:val_end], dates[val_end:]

print(f"📊 Data Split:")
print(f"  Train: {len(X_train)} samples")
print(f"  Val:   {len(X_val)} samples")
print(f"  Test:  {len(X_test)} samples")

## 5. 🧠 LSTM Model Architecture

We'll build a multi-output LSTM that predicts returns for all 5 days simultaneously.

In [ ]:
def build_lstm_model(input_shape, output_dim, units=[128, 64], dropout=0.2):
    """
    Build LSTM model for multi-horizon stock prediction.
    
    Architecture:
    - Bidirectional LSTM layers (capture forward and backward patterns)
    - Batch normalization (stabilize training)
    - Dropout (prevent overfitting)
    - Dense output layer (predict 5 days simultaneously)
    """
    model = Sequential([
        # Input layer
        Input(shape=input_shape),
        
        # First Bidirectional LSTM
        Bidirectional(LSTM(units[0], return_sequences=True)),
        BatchNormalization(),
        Dropout(dropout),
        
        # Second LSTM
        LSTM(units[1], return_sequences=False),
        BatchNormalization(),
        Dropout(dropout),
        
        # Dense layers
        Dense(32, activation='relu'),
        Dropout(dropout / 2),
        
        # Output: 5 predictions (day 1-5 returns)
        Dense(output_dim, activation='linear')
    ])
    
    return model

# Build model
input_shape = (SEQUENCE_LENGTH, len(FEATURE_COLS))
output_dim = len(PREDICTION_HORIZONS)  # 5 days

model = build_lstm_model(
    input_shape=input_shape,
    output_dim=output_dim,
    units=[128, 64],
    dropout=0.3
)

# Compile
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mae']
)

model.summary()

In [ ]:
# Callbacks
callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-6,
        verbose=1
    ),
    ModelCheckpoint(
        'best_lstm_model.keras',
        monitor='val_loss',
        save_best_only=True,
        verbose=0
    )
]

print("✅ Callbacks configured")

In [ ]:
# Train the model
print("🚂 Training LSTM model...")
print("="*60)

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=64,
    callbacks=callbacks,
    verbose=1
)

print("\n✅ Training complete!")

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss
axes[0].plot(history.history['loss'], label='Train Loss', color='#ff4444')
axes[0].plot(history.history['val_loss'], label='Val Loss', color='#44ff44')
axes[0].set_title('Model Loss', color='white')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss (MSE)')
axes[0].legend()
axes[0].grid(alpha=0.3)

# MAE
axes[1].plot(history.history['mae'], label='Train MAE', color='#ff4444')
axes[1].plot(history.history['val_mae'], label='Val MAE', color='#44ff44')
axes[1].set_title('Model MAE', color='white')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('MAE')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 6. 📊 Evaluation

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Predict on test set
y_pred_scaled = model.predict(X_test, verbose=0)

# Inverse transform to get actual returns
y_pred = target_scaler.inverse_transform(y_pred_scaled)
y_actual = y_test_raw

# Calculate metrics for each horizon
print("📊 Test Set Performance (per prediction horizon):")
print("="*70)

results = {}
for i, horizon in enumerate(PREDICTION_HORIZONS):
    pred = y_pred[:, i]
    actual = y_actual[:, i]
    
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    
    # Directional accuracy
    dir_acc = np.mean((pred > 0) == (actual > 0))
    
    results[horizon] = {
        'rmse': rmse,
        'mae': mae,
        'directional_accuracy': dir_acc
    }
    
    print(f"  Day {horizon}: RMSE={rmse:.6f}, MAE={mae:.6f}, Dir.Acc={dir_acc:.2%}")

# Average metrics
avg_rmse = np.mean([r['rmse'] for r in results.values()])
avg_mae = np.mean([r['mae'] for r in results.values()])
avg_dir_acc = np.mean([r['directional_accuracy'] for r in results.values()])

print("="*70)
print(f"\n🎯 Average: RMSE={avg_rmse:.6f}, MAE={avg_mae:.6f}, Dir.Acc={avg_dir_acc:.2%}")
print("\n(Directional accuracy > 50% is profitable in theory!)")

In [ ]:
# Visualize predictions vs actual
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

for i, horizon in enumerate(PREDICTION_HORIZONS):
    ax = axes[i]
    pred = y_pred[:, i] * 100  # Convert to percentage
    actual = y_actual[:, i] * 100
    
    ax.scatter(actual, pred, alpha=0.3, s=5, color='#ff4444')
    ax.plot([-10, 10], [-10, 10], 'w--', alpha=0.5, label='Perfect prediction')
    ax.set_xlabel('Actual Return (%)')
    ax.set_ylabel('Predicted Return (%)')
    ax.set_title(f'Day {horizon} Prediction\nDir.Acc: {results[horizon]["directional_accuracy"]:.2%}', color='white')
    ax.set_xlim(-10, 10)
    ax.set_ylim(-10, 10)
    ax.grid(alpha=0.3)
    ax.axhline(y=0, color='white', linewidth=0.5, alpha=0.5)
    ax.axvline(x=0, color='white', linewidth=0.5, alpha=0.5)

# Summary in last subplot
ax = axes[5]
horizons_str = [f'Day {h}' for h in PREDICTION_HORIZONS]
dir_accs = [results[h]['directional_accuracy'] * 100 for h in PREDICTION_HORIZONS]
bars = ax.bar(horizons_str, dir_accs, color='#ff4444')
ax.axhline(y=50, color='white', linestyle='--', label='Random (50%)')
ax.set_ylabel('Directional Accuracy (%)')
ax.set_title('Directional Accuracy by Horizon', color='white')
ax.set_ylim(0, 100)
ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Time series visualization for sample predictions
n_samples_to_show = 200

fig, axes = plt.subplots(len(PREDICTION_HORIZONS), 1, figsize=(16, 12), sharex=True)

for i, horizon in enumerate(PREDICTION_HORIZONS):
    ax = axes[i]
    
    pred = y_pred[-n_samples_to_show:, i] * 100
    actual = y_actual[-n_samples_to_show:, i] * 100
    
    ax.plot(pred, label='Predicted', color='#ff4444', linewidth=1)
    ax.plot(actual, label='Actual', color='#44ff44', linewidth=1, alpha=0.7)
    ax.axhline(y=0, color='white', linewidth=0.5, alpha=0.5)
    ax.set_ylabel(f'Day {horizon} Return (%)')
    ax.legend(loc='upper right')
    ax.grid(alpha=0.3)

axes[0].set_title('LSTM Predictions vs Actual Returns (Last 200 Samples)', color='white', fontsize=14)
axes[-1].set_xlabel('Sample Index')

plt.tight_layout()
plt.show()

## 7. 💾 Export for Production

In [ ]:
import json
import joblib
import os

os.makedirs('models_lstm', exist_ok=True)

# Save Keras model
model.save('models_lstm/lstm_price_predictor.keras')
print("✅ Saved models_lstm/lstm_price_predictor.keras")

# Save scalers
joblib.dump(feature_scaler, 'models_lstm/feature_scaler.pkl')
joblib.dump(target_scaler, 'models_lstm/target_scaler.pkl')
print("✅ Saved scalers")

# Save config
config = {
    'model_type': 'LSTM',
    'sequence_length': SEQUENCE_LENGTH,
    'feature_columns': FEATURE_COLS,
    'prediction_horizons': PREDICTION_HORIZONS,
    'selected_stocks': selected_stocks,
    'results': {str(k): {kk: float(vv) for kk, vv in v.items()} for k, v in results.items()},
    'avg_directional_accuracy': float(avg_dir_acc),
    'created_at': datetime.now().isoformat()
}

with open('models_lstm/config.json', 'w') as f:
    json.dump(config, f, indent=2)
print("✅ Saved models_lstm/config.json")

print("\n📦 All LSTM models exported!")

In [ ]:
# Download (Colab)
from google.colab import files

!zip -r models_lstm.zip models_lstm/
files.download('models_lstm.zip')

print("\n📥 Download complete!")

## 8. 🔧 Inference Class (for Backend)

In [ ]:
# === COPY THIS TO YOUR BACKEND ===

class LSTMPricePredictor:
    """
    Production LSTM price predictor for BVMT stocks.
    
    Usage:
        predictor = LSTMPricePredictor('models_lstm/')
        prediction = predictor.predict('SFBT', historical_features)
    """
    
    def __init__(self, model_dir: str):
        import json
        import joblib
        from tensorflow import keras
        
        with open(f'{model_dir}/config.json', 'r') as f:
            self.config = json.load(f)
        
        self.model = keras.models.load_model(f'{model_dir}/lstm_price_predictor.keras')
        self.feature_scaler = joblib.load(f'{model_dir}/feature_scaler.pkl')
        self.target_scaler = joblib.load(f'{model_dir}/target_scaler.pkl')
        
        print(f"✅ Loaded LSTM model")
    
    def predict(self, stock_name: str, historical_data: pd.DataFrame) -> dict:
        """
        Predict next 5 days prices.
        
        Args:
            stock_name: Stock name
            historical_data: DataFrame with at least SEQUENCE_LENGTH rows
                            and all required feature columns
        
        Returns:
            Prediction dictionary with prices for days 1-5
        """
        import numpy as np
        
        seq_len = self.config['sequence_length']
        feature_cols = self.config['feature_columns']
        
        if len(historical_data) < seq_len:
            raise ValueError(f"Need at least {seq_len} rows of data")
        
        # Get last seq_len rows
        recent = historical_data.tail(seq_len)
        current_price = recent['close'].iloc[-1]
        
        # Extract features
        X = recent[feature_cols].values
        
        # Scale
        X_scaled = self.feature_scaler.transform(X)
        X_input = X_scaled.reshape(1, seq_len, len(feature_cols))
        
        # Predict
        pred_scaled = self.model.predict(X_input, verbose=0)
        pred_returns = self.target_scaler.inverse_transform(pred_scaled)[0]
        
        # Build response
        predictions = {}
        for i, horizon in enumerate(self.config['prediction_horizons']):
            ret = pred_returns[i]
            pred_price = current_price * (1 + ret)
            
            predictions[f'day_{horizon}'] = {
                'predicted_price': round(pred_price, 3),
                'predicted_return': round(ret * 100, 2),
                'direction': 'UP' if ret > 0 else 'DOWN'
            }
        
        avg_return = np.mean(pred_returns) * 100
        if avg_return > 1.5:
            recommendation = 'BUY'
        elif avg_return < -1.5:
            recommendation = 'SELL'
        else:
            recommendation = 'HOLD'
        
        return {
            'stock': stock_name,
            'model': 'LSTM',
            'current_price': current_price,
            'predictions': predictions,
            'recommendation': recommendation,
            'avg_5d_return': round(avg_return, 2)
        }

print("✅ LSTMPricePredictor class ready!")

## 9. 📋 Summary

### Model Architecture:
- **Input**: 60-day sequences of 25 features
- **Layers**: Bidirectional LSTM(128) → LSTM(64) → Dense(32) → Dense(5)
- **Output**: 5 predicted returns (day 1-5)

### Key Differences from XGBoost:
| Aspect | XGBoost | LSTM |
|--------|---------|------|
| Input | Single feature vector | Sequence of 60 days |
| Training | Fast (~1 min) | Slower (~5-10 min) |
| Temporal patterns | Implicit via lagged features | Explicit via memory cells |
| Interpretability | Feature importance | Harder to interpret |

### Files Generated:
```
models_lstm/
├── lstm_price_predictor.keras
├── feature_scaler.pkl
├── target_scaler.pkl
└── config.json
```

---

# 🎉 LSTM Notebook Complete!

**IHEC-CODELAB 2.0**